In [110]:
#Imports
import requests
import json
import re
import pandas as pd
import tqdm

In [134]:
#Grabbing the list of CPG sites found in the Illumina human methylation 27 assay, so we can use both 27 and 450 assays
files_endpt = "https://api.gdc.cancer.gov/files"
filters = {
    "op": "and",
    "content":[
        {
            "op":"in",
            "content":{
                "field":"files.data_category",
                "value":"DNA Methylation"
            }
        },
        {
            "op": "in",
            "content":{
                "field": "cases.project.project_id",
                "value": "TCGA-BRCA"
                }
        },
                {
            "op": "in",
            "content":{
                "field": "files.platform",
                "value": "illumina human methylation 27"
                }
        }
    ]
}
params = {
    "filters": json.dumps(filters),
    "fields": "file_id",
    "format": "JSON",
    "size": "1"
    }
#Running request for file ids
response = requests.get(files_endpt, params = params)
#Parsing out file ids
file_uuid_list = []
for file_entry in json.loads(response.content.decode("utf-8"))["data"]["hits"]:
    file_uuid_list.append(file_entry["file_id"])
#Retrieving data using file ids
data_endpt = "https://api.gdc.cancer.gov/data"
params = {"ids": file_uuid_list}
#for file_id in file_uuid_list:
response = requests.post(data_endpt, data = json.dumps(params), headers = {"Content-Type": "application/json"})
response_head_cd = response.headers["Content-Disposition"]
file_name = './temp/temp_methyl.txt'
with open(file_name, "wb") as output_file:
    output_file.write(response.content)
    f = open('./temp/temp_methyl.txt','r')
    df = pd.read_csv('./temp/temp_methyl.txt', sep='\t')
    assay_27= df['Composite Element REF'].to_list()

In [214]:
#Request Body
#Only pulling methylation 450 files

files_endpt = "https://api.gdc.cancer.gov/files"
filters = {
    "op": "and",
    "content":[
        {
            "op":"in",
            "content":{
                "field":"files.data_category",
                "value":"DNA Methylation"
            }
        },
        {
            "op": "in",
            "content":{
                "field": "cases.project.project_id",
                "value": ["TCGA-BRCA","TCGA-GBM","TCGA-OV","TCGA-LUAD","TCGA-UCEC","TCGA-KIRC","TCGA-HNSC","TCGA-LGG"
                          ,"TCGA-THCA"
                          ,"TCGA-LUSC"
                          ,"TCGA-PRAD"
                          ,"TCGA-SKCM"
                          ,"TCGA-COAD"
                          ,"TCGA-STAD"
                          ,"TCGA-BLCA"
                          ,"TCGA-LIHC"
                          ,"TCGA-CESC"
                          ,"TCGA-KIRP"
                          ,"TCGA-SARC"
                          ,"TCGA-LAML"
                          ,"TCGA-ESCA"
                          ,"TCGA-PAAD"
                          ,"TCGA-PCPG"
                          ,"TCGA-READ"                          
                          ,"TCGA-TGCT"
                          ,"TCGA-THYM"
                          ,"TCGA-KICH"
                          ,"TCGA-ACC"
                          ,"TCGA-MESO"
                          ,"TCGA-UVM"
                          ,"TCGA-DLBC"
                          ,"TCGA-UCS"
                          ,"TCGA-CHOL"                          
                         ]
                }
        },
                {
            "op": "in",
            "content":{
                "field": "files.platform",
                "value": ["illumina human methylation 27"]
                }
        }
    ]
}
params = {
    "filters": json.dumps(filters),
    "fields": "file_id",
    "format": "JSON",
    "size": "20000"
    }

In [215]:
#Running request for file ids
response = requests.get(files_endpt, params = params)

In [216]:
#Parsing out file ids
file_uuid_list = []
for file_entry in json.loads(response.content.decode("utf-8"))["data"]["hits"]:
    file_uuid_list.append(file_entry["file_id"])

In [ ]:
#Retrieving data using file ids
data_endpt = "https://api.gdc.cancer.gov/data"
#params = {"ids": file_uuid_list}
for file_id in tqdm.tqdm(file_uuid_list):
    params = {"ids": [file_id]}
    response = requests.post(data_endpt, data = json.dumps(params), headers = {"Content-Type": "application/json"})
    response_head_cd = response.headers["Content-Disposition"]
    file_name = './temp/temp_methyl.txt'
    with open(file_name, "wb") as output_file:
        output_file.write(response.content)
        f = open('./temp/temp_methyl.txt','r')
        df = pd.read_csv('./temp/temp_methyl.txt', sep='\t')
        df = df.set_index('Composite Element REF')
        df = df.reindex(index = assay_27)
        df = df.reset_index()
        df2 = df.drop(['Chromosome','Composite Element REF','Start','End','Transcript_ID','Position_to_TSS','CGI_Coordinate','Feature_Type'],axis=1).dropna()
        df2 = df2[df2["Gene_Symbol"] != '.']
        df2 = df2[df2["Gene_Type"] != '.']
        df2 = df2.reset_index().drop(['index'],axis=1)
        gene_set = set()
        for genes in df2["Gene_Symbol"].to_list():
            for gene in genes.split(';'):
                gene_set.add(gene)
        df2['Gene_Symbol_Set'] = df2['Gene_Symbol'].apply(lambda x: ';'.join(list(set(x.split(';')))))
        df2 = df2.drop(['Gene_Symbol','Gene_Type'],axis=1)
        df3 = pd.DataFrame(index = gene_set,columns = ['beta_values'])
        for i in range(df2.shape[0]):
            for gene in df2.iloc[i]['Gene_Symbol_Set'].split(';'):        
                out = df3.loc[gene]
                if out.isna().to_list()[0]:
                    df3.loc[gene]['beta_values'] = str(df2.iloc[i]['Beta_value'])[:6]
                else:
                    df3.loc[gene]['beta_values'] = out['beta_values'] + ';' + str(df2.iloc[i]['Beta_value'])[:6]
        print(response_head_cd.split('filename=')[1])
        df3.to_csv('./methyl_files_27/' + response_head_cd.split('filename=')[1])

  0%|          | 1/2603 [00:18<13:10:28, 18.23s/it]

jhu-usc.edu_KIRP.HumanMethylation27.1.lvl-3.TCGA-B9-4113-11A-01D-1191-05.gdc_hg38.txt


  0%|          | 2/2603 [00:35<12:51:30, 17.80s/it]

jhu-usc.edu_KIRP.HumanMethylation27.1.lvl-3.TCGA-B3-4104-11A-01D-1191-05.gdc_hg38.txt


  0%|          | 3/2603 [00:52<12:42:33, 17.60s/it]

...

jhu-usc.edu_COAD.HumanMethylation27.7.lvl-3.TCGA-AA-3966-01A-01D-1110-05.gdc_hg38.txt


 16%|█▌        | 419/2603 [2:17:54<14:10:47, 23.37s/it]

jhu-usc.edu_COAD.HumanMethylation27.4.lvl-3.TCGA-AA-A01K-01A-01D-A00B-05.gdc_hg38.txt


 16%|█▌        | 420/2603 [2:18:16<13:51:16, 22.85s/it]

jhu-usc.edu_KIRC.HumanMethylation27.4.lvl-3.TCGA-CZ-4857-11A-01D-1303-05.gdc_hg38.txt
